## ¿Cómo construir en Python otros conjuntos númericos?

Existen muchos problemas de las ciencias computacionales que son más sencillos de resolver, si se cuenta con conjuntos númericos diferentes a $\mathbb{N}$, $\mathbb{Z}$, $\mathbb{Q}$, $\mathbb{R}$,  o $\mathbb{C}$. Por ejemplo, hay problemas que se modelan con grafos y en ocasiones la matrices de incidencia que describen a estas estructuras es mejor estudiarlas en un conjunto númerico como $\mathbb{Z}_2$. Entonces surge la pregunta ¿Cómo podemos construir en Python un conjuto númerico personalizado, en donde las operaciones clásicas de suma, resta, multiplicación y división las podamos definir a nuestro antojo? En Python, las clases cuenta con unos métodos especiales que se pueden redefinir y nos permite construir otros conjuntos númericos que se puede comportar de acuerdo a nuestra necesidad. Para entender cómo podemos hacer esto, voy a enseñarles como contruir una clase que nos permita hacer álgebra lineal utilizando el campo finito de $\mathbb{Z}_p$, en donde $p$ es un número primo.

Entonces para empezamos vamos a definir una clase para $\mathbb{Z}_p$ en donde vamos a redefinir las cuatro operaciones fundamentales, suma, resta, división y multiplicación, todas bajo el modulo de $p$, la estructura sería algo así:

In [114]:
from __future__ import annotations


class Zp:
    
    def __init__(self, prime: int = 2):
        self.prime = prime
        self.value = None
        
    def __call__(self, value: int) -> Zp:
        mod = self.__class__(self.prime)
        mod.value = value % self.prime
        return mod
        
    def __repr__(self) -> str:
        return str('{} (mod {})'.format(self.value, self.prime))

Observe que el valor de `prime` debe ser un número primo para que $\mathbb{Z}_{p}$ en efecto sea un campo númerico. Por el momento no vamos a validar que la variable de `prime` deberá ser agregada a criterio del usuario. 

Las clases tiene un serie de métodos con un significado especial. Por ejemplo, ya conocemos el significado del método `__init__(self, arg)` que se utiliza como constructur de clase, pero hay también muy interesantes, como los que usamos en la definición de la clase `Zp`, `__call__(self, args)` y `__repr__(self)`. En el transcurso de este post a hacer de otros más. Por el momento veamos porque usamos estos métodos.

`__call__` se utiliza para hacer que cada instancia de una clase se pueda llamas como un función, de modo que si se tiene una instancia como z2 = Zp(2), entonces podemos definer cada elemento de $\mathbb{Z}_{2}$ como `z2(1), z2(2), z2(3), ...`. En reliadad esto es un atajo a `z2.__call__(value)`. Veamos los resultados

In [115]:
mod2 = Zp(2)

En esta linea se ha instanciado el conjunto $\mathbb{Z}_{2}$, para hacer uso de sus elementos debemos hacer lo siguiente:

In [116]:
mod2(2)

0 (mod 2)

In [117]:
mod2(3)

1 (mod 2)

In [118]:
mod2(4)

0 (mod 2)

Acá la representación de $z2(n)$ se ha definido mediante el método `__repr__` , el cual permite convertir un objeto a una cadena, de mode que se puede llamar a la función interna `__repr__` sobre el objeto. Normalmente esta cadena tiene el aspecto de una expresión Python que podría usarse para recrear otro objeto con el mismo valor, o una cadena descriptiva en el caso de objetos complejos que podrían tener una representación complicada.  

Procedamos ahora a construir las cuatro operaciones fundamentales, suma, resta, división y multiplicación, todas bajo el modulo de 𝑝. Por ejemplo para la suma se define el método de la siguiente forma:

In [119]:
def __add__(self, other: Zn) -> Zn:
    if not self.prime == other.prime:
        raise AssertionError("Different fields ...")
    mod = self.__class__(self.prime)
    return mod(self.value + other.value) 

Es importante que el lector observe que para poder sumar dos valores $x, y$ estos deben pertenecer al mismo campo, en otras palabras que pertenezcan al mismo campo $\mathbb{Z}_p$ y de tal manera que la suma se define como $x + y\,\,(mod\,p)$.

Así nuestro clase quedaría así:

In [120]:
from __future__ import annotations


class Zn:
    def __init__(self, prime: int = 2):
        self.prime = prime
        self.value = None
        
    def __call__(self, value: int) -> Zn:
        mod = self.__class__(self.prime)
        mod.value = value % self.prime
        return mod
        
    def __repr__(self) -> str:
        return str('{} (mod {})'.format(self.value, self.prime))
    
    def __add__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value + other.value)

Veamos si la suma está funcionando bien, por ejemplo tomemos $3, 7 \in \mathbb{Z}_4$ y $3, 7 \in \mathbb{Z}_5$, y calculemos su suma:

In [127]:
mod4 = Zn(4)
mod5 = Zn(5)

In [128]:
a = mod4(3)
b = mod4(7)

In [129]:
a + b

2 (mod 4)

In [130]:
c = mod5(3)
d = mod5(7)

In [131]:
c + d

0 (mod 5)

¿Y que ocurre cuando se trata de suma $7\in \mathbb{Z}_4$ y $7\in \mathbb{Z}_7$?. Pues veamoslo:  

In [132]:
b + c

AssertionError: Different fields ...

Es justamente lo que estabamos esperando, no se pueden sumar números que pertenecen a conjuntos númericos diferentes.  

Como se puedo apreciar, el método `__add__(self, other)` permite definir como se va a usar el operador $+$, el primer parámetro es el primer operando de la suma, y el segudno parámetro es el segundo operando. Esto devuelve una nueva instancia, nunca modifica la clase actual. De la misma forma los métodos `__sub__(self, other)`, `__mul__(self, other)` y `__truediv__(self, other)` permiter redefinir los operadores $-$, $\times$ y $/$.

Por lo tanto usando los métodos `__sub__` y `__mul__` se define la sustracción y la multiplicación para $\mathbb{Z}_p$. En cuanto a la división se define una función `__inv_mul(self)` que mediante el algoritmo de Euclides posibilita calcular el inverso multiplitivo de cualquier elemento de $\mathbb{Z}_p$, y haciendo uso de esta función se redefine la división usando el método `__truediv__`, de tal manera que nuestra clase quedaría así:

In [133]:
from __future__ import annotations


class Zn:
    def __init__(self, prime: int = 2):
        self.prime = prime
        self.value = None
        
    def __call__(self, value: int) -> Zn:
        mod = self.__class__(self.prime)
        mod.value = value % self.prime
        return mod
        
    def __repr__(self) -> str:
        return str('{} (mod {})'.format(self.value, self.prime))
    
    def __add__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value + other.value)

    def __sub__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value - other.value) 
    
    def __mul__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value * other.value)
    
    def __truediv__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value * other.__inv__())
    
    def __inv__(self) -> int:
        if self.value == 0:
            raise ZeroDivisionError("The division for zero is not defined.")
        aux1 = 0
        aux2 = 1
        y = self.value
        x = self.prime
        while y != 0:
            q, r = divmod(x, y)
            x, y = y, r
            aux1, aux2 = aux2, (aux1 - q * aux2)
        if x == 1:
            return aux1 % self.prime
        else:
            raise AssertionError("There is not the multiplicative inverse.")

In [136]:
mod5 = Zn(5)

In [137]:
a = mod5(7)
b = mod5(13)

In [138]:
a

2 (mod 5)

In [139]:
b

3 (mod 5)

In [140]:
a - b

4 (mod 5)

In [141]:
a * b

1 (mod 5)

In [142]:
a / b

4 (mod 5)

In [143]:
mod6 = Zn(6)

In [144]:
mod6(100)

4 (mod 6)

Para finalizar vamos a sobrescribir los operadores `=+` y `=-` mediante los métodos `__iadd__` y `__iadd__`. También vamos a definir el inverso aditivo mediante el método especial `__neg__`. El resultado sería:

In [147]:
from __future__ import annotations


class Zn:
    def __init__(self, prime: int = 2):
        self.prime = prime
        self.value = None
        
    def __call__(self, value: int) -> Zn:
        mod = self.__class__(self.prime)
        mod.value = value % self.prime
        return mod
        
    def __repr__(self) -> str:
        return str('{} (mod {})'.format(self.value, self.prime))
    
    def __add__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value + other.value)
    
    def __iadd__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value)
    
    def __neg__(self) -> Zn:
        mod = self.__class__(self.prime)
        return mod(-self.value % self.prime)

    def __sub__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value - other.value)    
    
    def __isub__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value)
    
    def __mul__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value * other.value)
    
    def __truediv__(self, other: Zn) -> Zn:
        if not self.prime == other.prime:
            raise AssertionError("Different fields ...")
        mod = self.__class__(self.prime)
        return mod(self.value * other.__inv__())
    
    def __inv__(self) -> int:
        if self.value == 0:
            raise ZeroDivisionError("The division for zero is not defined.")
        aux1 = 0
        aux2 = 1
        y = self.value
        x = self.prime
        while y != 0:
            q, r = divmod(x, y)
            x, y = y, r
            aux1, aux2 = aux2, (aux1 - q * aux2)
        if x == 1:
            return aux1 % self.prime
        else:
            raise AssertionError("There is not the multiplicative inverse.")

In [149]:
import numpy as np


def add_mod2(x, y):
    return np.add(x, y) % 2


def mul_mod2(x, y):
    return  np.multiply(x, y) % 2
old_funcs = np.set_numeric_ops(add=add_mod2, multiply=mul_mod2)

<ipython-input-149-0e29c4bf9af7>:10: DeprecationWarning: PyArray_SetNumericOps is deprecated. Use PyUFunc_ReplaceLoopBySignature to replace ufunc inner loop functions instead.
  old_funcs = np.set_numeric_ops(add=add_mod2, multiply=mul_mod2)


In [ ]:
http://scielo.sld.cu/scielo.php?script=sci_arttext&pid=S2227-18992020000200098

In [11]:
-3%4

1